<a href="https://colab.research.google.com/github/stellagerantoni/learning-time-series-counterfactuals/blob/main/multivariate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
 ! git clone https://github.com/stellagerantoni/learning-time-series-counterfactuals
 %cd learning-time-series-counterfactuals/

Cloning into 'learning-time-series-counterfactuals'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 312 (delta 122), reused 24 (delta 24), pack-reused 138
Receiving objects: 100% (312/312), 4.38 MiB | 12.08 MiB/s, done.
Resolving deltas: 100% (191/191), done.
/content/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals


In [83]:

!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon[all_extras]

In [84]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals

from _composite import ModifiedLatentCF
%cd src
from _vanilla import LatentCF
from help_functions import (ResultWriter, conditional_pad, evaluate,
                            find_best_lr, plot_graphs,
                            reset_seeds, time_series_normalize,
                            time_series_revert, upsample_minority,
                            validity_score)
from keras_models import *

/content/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src/learning-time-series-counterfactuals/src


In [85]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

In [86]:

X, y, meta_data = load_classification("Heartbeat")
print(" Shape of X = ", X.shape)
print(" Meta data = ", meta_data)
print(X[:3])

 Shape of X =  (409, 61, 405)
 Meta data =  {'problemname': 'heartbeat', 'timestamps': False, 'missing': False, 'univariate': False, 'equallength': True, 'classlabel': True, 'targetlabel': False, 'class_values': ['normal', 'abnormal']}
[[[9.4900e-04 1.4880e-03 3.1400e-04 ... 8.0400e-04 8.1500e-04 1.3890e-03]
  [1.2880e-03 1.1400e-03 4.3000e-04 ... 1.3640e-03 3.6300e-04 8.4300e-04]
  [5.2900e-04 1.6350e-03 2.1460e-03 ... 1.1790e-03 3.6300e-04 5.0400e-04]
  ...
  [7.8312e-02 1.4568e-01 2.3805e-01 ... 4.4070e-01 6.3277e-01 6.5026e-01]
  [4.5608e-02 1.1980e-01 1.7813e-01 ... 3.7934e-01 3.7387e-01 3.2700e-01]
  [1.2107e-01 1.3385e-01 4.0077e-02 ... 9.8976e-02 2.5955e-02 3.6082e-02]]

 [[2.0264e-02 1.7023e-02 6.0520e-03 ... 5.3840e-03 8.3720e-03 7.8450e-03]
  [2.7680e-02 3.6928e-02 3.4255e-02 ... 1.5869e-02 8.6480e-03 2.5470e-03]
  [2.3423e-02 4.1957e-02 3.4538e-02 ... 9.0860e-03 9.5190e-03 7.3490e-03]
  ...
  [7.8027e-01 4.4498e-01 4.9525e-01 ... 9.8392e-02 2.0994e-01 4.6098e-01]
  [5.4370e

In [87]:
pos = 'normal'
neg = 'abnormal'
# Convert positive and negative labels to 1 and 0
pos_label, neg_label = 1, 0
y_copy = y.copy()
if pos != pos_label:
    y_copy[y==pos] = pos_label # convert/normalize positive label to 1
if neg != neg_label:
    y_copy[y==neg] = neg_label # convert negative label to 0

y_copy = y_copy.astype(int)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y_copy, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [89]:
pos_counts = pd.value_counts(y)[pos_label]
neg_counts = pd.value_counts(y)[neg_label]

In [90]:
print(pos_counts)
print(neg_counts)

114
295


In [91]:
from sklearn.utils import resample, shuffle
import numpy as np
import pandas as pd

def downsample_multivariate_time_series(X, y, pos_label=1, neg_label=0, random_state=39):
    # Get counts
    pos_counts = pd.value_counts(y)[pos_label]
    neg_counts = pd.value_counts(y)[neg_label]

    # Divide by class
    X_pos, X_neg = X[y == pos_label], X[y == neg_label]

    if pos_counts == neg_counts:
        # Balanced dataset
        return X, y
    elif pos_counts < neg_counts:
        # Downsample majority class
        X_neg_down = resample(
            X_neg, replace=False, n_samples=pos_counts, random_state=random_state
        )
        X_concat = np.concatenate([X_pos, X_neg_down], axis=0)
        y_concat = np.array(
            [pos_label for i in range(pos_counts)]
            + [neg_label for j in range(pos_counts)]
        )
    else:
        # Downsample majority class
        X_pos_down = resample(
            X_pos, replace=False, n_samples=neg_counts, random_state=random_state
        )
        X_concat = np.concatenate([X_pos_down, X_neg], axis=0)
        y_concat = np.array(
            [pos_label for i in range(neg_counts)]
            + [neg_label for j in range(neg_counts)]
        )

    # Shuffle the index after down-sampling
    X_concat, y_concat = shuffle(X_concat, y_concat, random_state=random_state)

    return X_concat, y_concat

In [92]:
def downsample_majority(X, y, pos_label=1, neg_label=0, random_state=39):
    # Get counts
    pos_counts = pd.value_counts(y)[pos_label]
    neg_counts = pd.value_counts(y)[neg_label]

    # Divide by class
    X_pos, X_neg = X[y == pos_label], X[y == neg_label]

    if pos_counts == neg_counts:
        # Balanced dataset
        return X, y
    elif pos_counts < neg_counts:
        # Downsample majority class
        X_neg_down = resample(
            X_neg, replace=False, n_samples=pos_counts, random_state=random_state
        )
        X_concat = np.concatenate([X_pos, X_neg_down], axis=0)
        y_concat = np.array(
            [pos_label for i in range(pos_counts)]
            + [neg_label for j in range(pos_counts)]
        )
    else:
        # Downsample majority class
        X_pos_down = resample(
            X_pos, replace=False, n_samples=neg_counts, random_state=random_state
        )
        X_concat = np.concatenate([X_pos_down, X_neg], axis=0)
        y_concat = np.array(
            [pos_label for i in range(neg_counts)]
            + [neg_label for j in range(neg_counts)]
        )

    # Shuffle the index after down-sampling
    X_concat, y_concat = shuffle(X_concat, y_concat, random_state=random_state)

    return X_concat, y_concat

In [93]:
from sklearn.utils import resample, shuffle
import numpy as np
import pandas as pd

def upsample_multivariate_time_series(X, y, pos_label=1, neg_label=0, random_state=39):
    # Ensure that the data is in the right format
    if len(X.shape) != 3:
        raise ValueError("X should be a 3D array: [samples, features, time_steps].")

    # Convert y to numpy array for indexing
    y = np.array(y)

    # Get counts
    pos_counts = np.sum(y == pos_label)
    neg_counts = np.sum(y == neg_label)

    # Split data by class
    X_pos, X_neg = X[y == pos_label], X[y == neg_label]

    if pos_counts == neg_counts:
        # Balanced dataset
        return X, y
    elif pos_counts > neg_counts:
        # Imbalanced dataset
        X_neg_over = resample(
            X_neg, replace=True, n_samples=pos_counts, random_state=random_state
        )
        X_concat = np.concatenate([X_pos, X_neg_over], axis=0)
        y_concat = np.array(
            [pos_label for i in range(pos_counts)]
            + [neg_label for j in range(pos_counts)]
        )
    else:
        # Imbalanced dataset
        X_pos_over = resample(
            X_pos, replace=True, n_samples=neg_counts, random_state=random_state
        )
        X_concat = np.concatenate([X_pos_over, X_neg], axis=0)
        y_concat = np.array(
            [pos_label for i in range(neg_counts)]
            + [neg_label for j in range(neg_counts)]
        )

    # Shuffle the upsampled dataset
    shuffled_indices = np.arange(X_concat.shape[0])
    np.random.shuffle(shuffled_indices)

    X_concat = X_concat[shuffled_indices]
    y_concat = y_concat[shuffled_indices]

    return X_concat, y_concat

In [94]:
# Upsample the minority class

pos_counts = pd.value_counts(y_train)[pos_label]
neg_counts = pd.value_counts(y_train)[neg_label]
print(f"negative_count = {neg_counts}, positive_count = {pos_counts}")

if pos_counts!=neg_counts:
  X_train, y_train = downsample_majority(X_train, y_train, pos_label=pos_label, neg_label=neg_label)
  print(f"Data upsampling performed, current distribution of y: \n{pd.value_counts(y_train)}.")
else:
   print(f"Data upsampling not needed, current distribution of y: \n{pd.value_counts(y_train)}.")


negative_count = 236, positive_count = 91
Data upsampling performed, current distribution of y: 
1    91
0    91
dtype: int64.


In [96]:
pos_counts = pd.value_counts(y_train)[pos_label]
neg_counts = pd.value_counts(y_train)[neg_label]

In [97]:
print(pos_counts)
neg_counts

91


91

In [98]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def time_series_normalize_multivariate(data, n_timesteps, n_features, scaler=None):
    # First transpose the data to have shape (samples, timesteps, features)
    data_transposed = np.transpose(data, (0, 2, 1))

    # Then reshape data to have timesteps as rows for normalization
    data_reshaped = data_transposed.reshape(-1, n_features)

    if scaler is None:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(data_reshaped)

    normalized = scaler.transform(data_reshaped)

    # Return data reshaped
    normalized_transposed = normalized.reshape(-1, n_timesteps, n_features)
    return np.transpose(normalized_transposed, (0, 2, 1)), scaler

In [99]:
def conditional_pad_multivariate(X):
    num_timesteps = X.shape[2]

    if num_timesteps % 4 != 0:
        next_num = (int(num_timesteps / 4) + 1) * 4
        padding_size = next_num - num_timesteps
        X_padded = np.pad(
            X, pad_width=((0, 0), (0, 0), (0, padding_size))
        )

        return X_padded, padding_size

    return X, 0

In [100]:
n_training, n_features, n_timesteps = X_train.shape

X_train_processed, trained_scaler =  time_series_normalize_multivariate(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ =  time_series_normalize_multivariate(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X_train_processed_padded, padding_size = conditional_pad_multivariate(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad_multivariate(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[2]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

Data pre-processed, original #timesteps=405, padded #timesteps=408.


In [101]:
#check that padding paddes the right dimention
# The timesteps(3rd dimention) should have changed if padded was needed
print(f"X_train.shape = {X_train.shape}" )
print(f"X_train_processed_padded.shape = {X_train_processed_padded.shape}")

X_train.shape = (182, 61, 405)
X_train_processed_padded.shape = (182, 61, 408)


In [104]:
#check the processing (0,1) min should be min 0 and max should be max 1
print(f"min value = {np.min(X_train)}, max value = {np.max(X_train)}")
print(f"min value normalized = {np.min(X_train_processed)}, max value normalized= {np.max(X_train_processed)}")

min value = 3e-06, max value = 35.986
min value normalized = 0.0, max value normalized= 1.0000000000000002


In [105]:
y_train_classes = y_train
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_test = to_categorical(y_test, len(np.unique(y_test)))

In [106]:
print(np.min(X_train_processed), np.max(X_train_processed))
print(np.min(X_train), np.max(X_train))

0.0 1.0000000000000002
3e-06 35.986


In [107]:
def Classifier(
    n_timesteps, n_features, n_conv_layers=1, add_dense_layer=True, n_output=1
):
    # https://keras.io/examples/timeseries/timeseries_classification_from_scratch/
    inputs = keras.Input(shape=(n_features, n_timesteps), dtype="float32")

    if add_dense_layer:
        x = keras.layers.Dense(128)(inputs)
    else:
        x = inputs

    for i in range(n_conv_layers):
        x = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)

    x = keras.layers.MaxPooling1D(pool_size=2, padding="same")(x)
    x = keras.layers.Flatten()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)

    return classifier

In [110]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

def Classifierl2(
    n_timesteps, n_features, n_conv_layers=1, add_dense_layer=True, n_output=1, reg_rate=0.01, dropout_rate=0.5
):
    # https://keras.io/examples/timeseries/timeseries_classification_from_scratch/
    inputs = keras.Input(shape=(n_features, n_timesteps), dtype="float32")

    if add_dense_layer:
        x = keras.layers.Dense(128, kernel_regularizer=l2(reg_rate))(inputs)
    else:
        x = inputs

    for i in range(n_conv_layers):
        x = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", kernel_regularizer=l2(reg_rate))(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)
        x = Dropout(dropout_rate)(x)  # Dropout after each convolutional block

    x = keras.layers.MaxPooling1D(pool_size=2, padding="same")(x)
    x = keras.layers.Flatten()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax", kernel_regularizer=l2(reg_rate))(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(reg_rate))(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)
    return classifier

In [113]:
pos_counts = pd.value_counts(y_train_classes)[pos_label]
neg_counts = pd.value_counts(y_train_classes)[neg_label]

In [115]:
print(pos_counts)
print(neg_counts)

91
91


In [116]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

def SimpleClassifier(
    n_timesteps, n_features, n_output=1, reg_rate=0.01, dropout_rate=0.3
):
    inputs = keras.Input(shape=(n_features, n_timesteps), dtype="float32")

    # One convolutional layer
    x = keras.layers.Conv1D(filters=32, kernel_size=3, padding="same", kernel_regularizer=l2(reg_rate))(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = Dropout(dropout_rate)(x)  # Lower the dropout rate a bit

    x = keras.layers.MaxPooling1D(pool_size=2, padding="same")(x)
    x = keras.layers.Flatten()(x)

    # Simpler dense layer
    x = keras.layers.Dense(64, activation="relu", kernel_regularizer=l2(reg_rate))(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax", kernel_regularizer=l2(reg_rate))(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(reg_rate))(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)
    return classifier





In [126]:
shallow_cnn = True
# ### 1dCNN classifier
if shallow_cnn == True:
    print(f"Check shallow_cnn argument={shallow_cnn}, use the shallow structure.")
    classifier = Classifier(n_timesteps_padded, n_features) # shallow CNN for small data size
else:
    classifier = Classifierl2(n_timesteps_padded, n_features, n_conv_layers=3, add_dense_layer=False) # deeper CNN layers for data with larger size

optimizer = keras.optimizers.Adam(lr=0.0001)
classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)
# Train the model
reset_seeds()
classifier_history = classifier.fit(X_train_processed_padded,
        y_train_classes,
        epochs=150,
        batch_size=32,
        shuffle=True,
        verbose=True,
        validation_data=(X_test_processed_padded, y_test_classes),
        callbacks=[early_stopping_accuracy])

Check shallow_cnn argument=True, use the shallow structure.
Epoch 1/150
6/6 [==============================] - 1s 55ms/step - loss: 0.8310 - accuracy: 0.5220 - val_loss: 0.6917 - val_accuracy: 0.5610
Epoch 2/150
6/6 [==============================] - 0s 24ms/step - loss: 0.5511 - accuracy: 0.7473 - val_loss: 0.6865 - val_accuracy: 0.7683
Epoch 3/150
6/6 [==============================] - 0s 23ms/step - loss: 0.4841 - accuracy: 0.7912 - val_loss: 0.6871 - val_accuracy: 0.6341
Epoch 4/150
6/6 [==============================] - 0s 21ms/step - loss: 0.4164 - accuracy: 0.8352 - val_loss: 0.6807 - val_accuracy: 0.7561
Epoch 5/150
6/6 [==============================] - 0s 23ms/step - loss: 0.3946 - accuracy: 0.8242 - val_loss: 0.6781 - val_accuracy: 0.7561
Epoch 6/150
6/6 [==============================] - 0s 25ms/step - loss: 0.3420 - accuracy: 0.9066 - val_loss: 0.6767 - val_accuracy: 0.7439
Epoch 7/150
6/6 [==============================] - 0s 22ms/step - loss: 0.3132 - accuracy: 0.9286 - 

In [127]:
y_pred = classifier.predict(X_test_processed_padded)
# y_pred_classes = np.array([1 if pred > 0.5 else 0 for pred in y_pred])
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"1dCNN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
        index=['True:pos', 'True:neg'],
        columns=['Pred:pos', 'Pred:neg']
    )
confusion_matrix_df

3/3 [==============================] - 0s 5ms/step
1dCNN classifier trained, with validation accuracy 0.5.


,Pred:pos,Pred:neg
True:pos,0,23
True:neg,0,59
